In [7]:
from ycharts_parsers import parse_html_to_pd, YchartsDataVar
from os import listdir

YCH_DATA_TYPE = YchartsDataVar.TOTAL_ASSETS
PARQUETS_PATH = f"./data/ycharts/{YCH_DATA_TYPE.value}_parqs"

In [8]:
len(listdir(PARQUETS_PATH))

2139